# Privacy attack simulation with Iris dataset

## Installing dependencies

In [ ]:
!pip install pandas

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets 

In [ ]:
iris = datasets.load_iris()

## Testing dataset with Pandas

In [ ]:
data1 = pd.DataFrame(data = np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])

## Splitting data

In [ ]:
from sklearn.model_selection import train_test_split
x=iris.data
y=iris.target
x_a,x_b,y_a,y_b=train_test_split(x,y,test_size=.5)
x_b1,x_b2,y_b1,y_b2=train_test_split(x_b,y_b,test_size=.2)

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_b1,y_b1)

y_pred=clf.predict(x_b2)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_b2, y_pred))